# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 7 2022 8:50AM,238954,10,LAPS-705080,"Herbivore Botanicals, LLC.",Released
1,Jun 7 2022 8:48AM,238953,10,Eme-100258,Emerginnova,Released
2,Jun 7 2022 8:48AM,238953,10,Enova-10090,Emerginnova,Released
3,Jun 7 2022 8:48AM,238953,10,Enova-10091,Emerginnova,Released
4,Jun 7 2022 8:48AM,238953,10,Eme-100262,Emerginnova,Released
5,Jun 7 2022 8:44AM,238951,10,0085741068,ISDIN Corporation,Released
6,Jun 7 2022 8:44AM,238951,10,0085741067,ISDIN Corporation,Released
7,Jun 7 2022 8:44AM,238951,10,0085741069,ISDIN Corporation,Released
8,Jun 7 2022 8:44AM,238951,10,0085741091,ISDIN Corporation,Released
9,Jun 7 2022 8:44AM,238951,10,0085741107,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,238948,Released,2
38,238950,Cancelled,1
39,238951,Released,14
40,238953,Released,4
41,238954,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238948,NaN,NaN,NaN,2.0
238950,1.0,NaN,NaN,NaN
238951,NaN,NaN,NaN,14.0
238953,NaN,NaN,NaN,4.0
238954,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238844,0.0,0.0,0.0,3.0
238857,0.0,0.0,1.0,0.0
238868,0.0,0.0,0.0,1.0
238873,0.0,0.0,0.0,1.0
238879,0.0,0.0,5.0,39.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238844,0.0,0,0,3
238857,0.0,0,1,0
238868,0.0,0,0,1
238873,0.0,0,0,1
238879,0.0,0,5,39


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,238844,0.0,0,0,3
1,238857,0.0,0,1,0
2,238868,0.0,0,0,1
3,238873,0.0,0,0,1
4,238879,0.0,0,5,39


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,238844,0.0,,,3
1,238857,0.0,,1,
2,238868,0.0,,,1
3,238873,0.0,,,1
4,238879,0.0,,5,39


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 7 2022 8:50AM,238954,10,"Herbivore Botanicals, LLC."
1,Jun 7 2022 8:48AM,238953,10,Emerginnova
5,Jun 7 2022 8:44AM,238951,10,ISDIN Corporation
19,Jun 7 2022 8:38AM,238950,10,"Citieffe, Inc."
20,Jun 7 2022 7:59AM,238947,10,"Citieffe, Inc."
21,Jun 7 2022 7:51AM,238948,19,"AdvaGen Pharma, Ltd"
23,Jun 7 2022 7:34AM,238946,19,"AdvaGen Pharma, Ltd"
24,Jun 7 2022 12:00AM,238924,10,ISDIN Corporation
36,Jun 7 2022 12:00AM,238938,10,"Citieffe, Inc."
70,Jun 7 2022 12:00AM,238934,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Jun 7 2022 8:50AM,238954,10,"Herbivore Botanicals, LLC.",0.0,,,1
1,Jun 7 2022 8:48AM,238953,10,Emerginnova,0.0,,,4
2,Jun 7 2022 8:44AM,238951,10,ISDIN Corporation,0.0,,,14
3,Jun 7 2022 8:38AM,238950,10,"Citieffe, Inc.",1.0,,,
4,Jun 7 2022 7:59AM,238947,10,"Citieffe, Inc.",0.0,,,1
5,Jun 7 2022 7:51AM,238948,19,"AdvaGen Pharma, Ltd",0.0,,,2
6,Jun 7 2022 7:34AM,238946,19,"AdvaGen Pharma, Ltd",0.0,,,1
7,Jun 7 2022 12:00AM,238924,10,ISDIN Corporation,0.0,,,44
8,Jun 7 2022 12:00AM,238938,10,"Citieffe, Inc.",0.0,,,2
9,Jun 7 2022 12:00AM,238934,10,Eywa Pharma Inc.,0.0,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 8:50AM,238954,10,"Herbivore Botanicals, LLC.",1,,
1,Jun 7 2022 8:48AM,238953,10,Emerginnova,4,,
2,Jun 7 2022 8:44AM,238951,10,ISDIN Corporation,14,,
3,Jun 7 2022 8:38AM,238950,10,"Citieffe, Inc.",,,
4,Jun 7 2022 7:59AM,238947,10,"Citieffe, Inc.",1,,
5,Jun 7 2022 7:51AM,238948,19,"AdvaGen Pharma, Ltd",2,,
6,Jun 7 2022 7:34AM,238946,19,"AdvaGen Pharma, Ltd",1,,
7,Jun 7 2022 12:00AM,238924,10,ISDIN Corporation,44,,
8,Jun 7 2022 12:00AM,238938,10,"Citieffe, Inc.",2,,
9,Jun 7 2022 12:00AM,238934,10,Eywa Pharma Inc.,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 8:50AM,238954,10,"Herbivore Botanicals, LLC.",1,,
1,Jun 7 2022 8:48AM,238953,10,Emerginnova,4,,
2,Jun 7 2022 8:44AM,238951,10,ISDIN Corporation,14,,
3,Jun 7 2022 8:38AM,238950,10,"Citieffe, Inc.",,,
4,Jun 7 2022 7:59AM,238947,10,"Citieffe, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 8:50AM,238954,10,"Herbivore Botanicals, LLC.",1.0,NaN,NaN
1,Jun 7 2022 8:48AM,238953,10,Emerginnova,4.0,NaN,NaN
2,Jun 7 2022 8:44AM,238951,10,ISDIN Corporation,14.0,NaN,NaN
3,Jun 7 2022 8:38AM,238950,10,"Citieffe, Inc.",NaN,NaN,NaN
4,Jun 7 2022 7:59AM,238947,10,"Citieffe, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 8:50AM,238954,10,"Herbivore Botanicals, LLC.",1.0,0.0,0.0
1,Jun 7 2022 8:48AM,238953,10,Emerginnova,4.0,0.0,0.0
2,Jun 7 2022 8:44AM,238951,10,ISDIN Corporation,14.0,0.0,0.0
3,Jun 7 2022 8:38AM,238950,10,"Citieffe, Inc.",0.0,0.0,0.0
4,Jun 7 2022 7:59AM,238947,10,"Citieffe, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2867115,76.0,2.0,11.0
12,238886,0.0,1.0,0.0
16,477814,8.0,18.0,0.0
17,477784,2.0,0.0,0.0
18,477771,2.0,0.0,0.0
19,1672379,14.0,11.0,0.0
20,1433385,90.0,28.0,0.0
21,716783,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2867115,76.0,2.0,11.0
1,12,238886,0.0,1.0,0.0
2,16,477814,8.0,18.0,0.0
3,17,477784,2.0,0.0,0.0
4,18,477771,2.0,0.0,0.0
5,19,1672379,14.0,11.0,0.0
6,20,1433385,90.0,28.0,0.0
7,21,716783,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,76.0,2.0,11.0
1,12,0.0,1.0,0.0
2,16,8.0,18.0,0.0
3,17,2.0,0.0,0.0
4,18,2.0,0.0,0.0
5,19,14.0,11.0,0.0
6,20,90.0,28.0,0.0
7,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,76.0
1,12,Released,0.0
2,16,Released,8.0
3,17,Released,2.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,18,19,20,21
Status,,,,,,,,
Completed,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,2.0,1.0,18.0,0.0,0.0,11.0,28.0,1.0
Released,76.0,0.0,8.0,2.0,2.0,14.0,90.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,18,19,20,21
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,2.0,1.0,18.0,0.0,0.0,11.0,28.0,1.0
2,Released,76.0,0.0,8.0,2.0,2.0,14.0,90.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,18,19,20,21
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,2.0,1.0,18.0,0.0,0.0,11.0,28.0,1.0
2,Released,76.0,0.0,8.0,2.0,2.0,14.0,90.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()